# Examples for modrinth-ureq Queries

The examples in this project are in a Jupyter notebook rather than cargo-runnable files because
I prefer this for testing personally, and don't want to maintain two forms of examples.

## How to Use

You will need the `evcxr` Jupyter kernel. Install it with the following command.

```bash
$ cargo install evcxr_repl evcxr_jupyter
```

Note that while `evcxr_repl` isn't necessary, you may play with it and find it useful.

**The cells in this notebook need to be run in order.**

Some are independant, but for the majority, proper order is top-down. Note that all examples
require the import in the first cell. Change the path of the dependency statement to match the location
of the `modrinth` crate root based on where you put the files.

Note that `evxcr` declarations (for dependencies for example) must come before comments.

In [ ]:
:dep modrinth = { path = "/home/jacob/Documents/github.com/spikespaz/umapacker/modrinth" }

use modrinth::prelude::*;

In [ ]:
:dep serde_json = { version = "*" }

// This simply shows how the `SearchFacet` variants are going to be serialized for requests.
// It also defines the variable `params` used in lower cells.

let params = SearchParams {
    query: None,
    facets: Some(
        vec![
            vec![
                SearchFacet::license("MIT"),
                SearchFacet::license("Apache"),
            ],
            vec![
                SearchFacet::category("fabric"),
            ],
            vec![
                SearchFacet::version("1.18"),
                SearchFacet::version("1.18.1"),
                SearchFacet::version("1.18.2"),
            ],
        ]
    ),
    index: None,
    offset: None,
    limit: None,
    filters: None,
};

println!("{}", serde_json::to_string_pretty(&params).unwrap());

In [ ]:
// Finally, this shows how the query parameters are going to be serialized with stardard URL encoding
// before sending the string off to the request library.

use modrinth::query_string::JsonQueryParams;

println!("{}", params.to_query_string());

In [ ]:
// Make a search using the parameters defined in the one of the previous cells.
// This will only show results up to the specified `limit` in the `SearchParams`,
// or it uses the default 10 specified by the API. The default is not hard-coded,
// so if the remote API changes the code here will use that new value.

println!("{:#?}", get_search(&params, None));

In [ ]:
// The above example shows only the default of 10 results.
// This one creates an iterator which handles the pagination for you, with a maximum
// limit of 100 results per page. This upper limit is not enforced by this crate,
// and using anything higher than what is specified as the maximum by the API
// will result in undefined behavior.

let mut iterator = get_search_iter(params.clone(), None);
let mut count = 0;

for result in &mut iterator {
    match result {
        Ok(result) => {
            count += 1;
            println!("{:?}", result);
        }
        Err(error) => println!("\nError: {:#?}", error),
    }
}

println!("\nExpected results: {}\nReceived results: {}", iterator.size_hint().1.unwrap(), count);

In [ ]:
// This is how you request a specific project. You can use a name/slug or a `Base62` identifier.

println!("{:#?}", get_project(&ProjectIdentifier::from("sodium"), None));

In [ ]:
// Here we get the versions available for a project by name.

println!("{:#?}", get_project_versions(&ProjectIdentifier::from("sodium"), None))